In [80]:
import pandas as pd

df = pd.read_csv('input.csv')
df

,class_name,line_number,AST
0,ApplicationInsightsForK8sHostingStartup,9,ClassDeclaration\n\t\t BaseList\n\t\t\t Simpl...
1,ApplicationInsightsForK8sHostingStartup,15,MethodDeclaration\n\t\t\t PredefinedType\n\t\...
2,ApplicationInsightsForK8sHostingStartup,16,Block
3,ApplicationInsightsForK8sHostingStartup,17,ExpressionStatement\n\t\t\t\t\t InvocationExp...
4,ApplicationInsightsForK8sHostingStartup,18,Block
...,...,...,...
27160,OpenApiWorkspaceTests,209,SimpleAssignmentExpression\n\t\t\t\t\t\t\t\t ...
27161,OpenApiWorkspaceTests,211,SimpleAssignmentExpression\n\t\t\t\t\t\t\t\t\...
27162,OpenApiWorkspaceTests,212,SimpleAssignmentExpression\n\t\t\t\t\t\t\t\t\...
27163,OpenApiWorkspaceTests,213,SimpleAssignmentExpression\n\t\t\t\t\t\t\t\t\...


## Cossim prediction

In [85]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import pickle

# Load the embeddings from parsed_smell_identifier_embeddings.pt
embeddings = torch.load('parsed_smell_identifier_embeddings.pt')

# Load the encoded messages from messagelist.pkl
with open('messagelist.pkl', 'rb') as f:
    message_list = pickle.load(f)

# Load the message encoding vocabulary
with open('message_encoding_vocabulary.pkl', 'rb') as f:
    message_encoding_vocabulary = pickle.load(f)

# Convert the embeddings into a suitable format for comparison
existing_embeddings = np.array(embeddings.tolist())

# Load the new ASTs from the CSV file
new_data = pd.read_csv('input.csv', nrows=100)

# Preprocess the new ASTs
new_ast_list = new_data['AST'].tolist()
preprocessed_ast_list = [ast.strip() for ast in new_ast_list]

# Convert the new ASTs into embeddings using the same sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
new_ast_embeddings = model.encode(preprocessed_ast_list)

# Classify the new ASTs based on similarity to existing embeddings
for i, new_embedding in enumerate(new_ast_embeddings):
    similarities = cosine_similarity([new_embedding], existing_embeddings)
    max_similarity = np.max(similarities)

    if max_similarity >= 0.95:
        predicted_index = np.argmax(similarities)
        predicted_message = message_encoding_vocabulary[y[predicted_index]]
        class_name = new_data.loc[i, 'class_name']
        line_number = new_data.loc[i, 'line_number']
        print("Belonged Message:", predicted_message)
        print("Class Name:", class_name)
        print("Line Number:", line_number)
        print("Similarity Score:", max_similarity)
        print()


Belonged Message: Rename
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 9
Similarity Score: 1.0000000000000009

Belonged Message: Refactor
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 15
Similarity Score: 0.9614033398409455

Belonged Message: Refactor
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 16
Similarity Score: 0.9999999999999132

Belonged Message: Refactor
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 17
Similarity Score: 0.9603806624511894

Belonged Message: Refactor
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 18
Similarity Score: 0.9999999999999132

Belonged Message: Refactor
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 19
Similarity Score: 0.9798968526812155

Belonged Message: Adjustment
Class Name: CGroupContainerIdProvider
Line Number: 9
Similarity Score: 0.9999999999999395

Belonged Message: Remove
Class Name: CGroupContainerIdProvider
Line Number: 10
Simila

# FNN

In [84]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import pickle

# Load the embeddings from parsed_smell_identifier_embeddings.pt
embeddings = torch.load('parsed_smell_identifier_embeddings.pt')

# Load the encoded messages from messagelist.pkl
with open('messagelist.pkl', 'rb') as f:
    message_list = pickle.load(f)

# Load the message encoding vocabulary
with open('message_encoding_vocabulary.pkl', 'rb') as f:
    message_encoding_vocabulary = pickle.load(f)
    

# Convert the embeddings into a suitable format for comparison
existing_embeddings = np.array(embeddings.tolist())

# Create a DataFrame with the embeddings and encoded messages
data = pd.DataFrame({
    'parsed_smell_identifier_embeddings': existing_embeddings.tolist(),
    'message_encoded': message_list
})

# Prepare the data for the neural network
X = np.array(data['parsed_smell_identifier_embeddings'].tolist())
y = np.array(data['message_encoded'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train).float()
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test).float()
y_test_tensor = torch.tensor(y_test)

# Define the neural network architecture
class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define the model parameters
input_dim = X_train_tensor.shape[1]
hidden_dim = 128
output_dim = len(np.unique(y_train))

# Create the neural network model
model = FeedforwardNN(input_dim, hidden_dim, output_dim)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader objects
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Training the neural network model
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluate the neural network model
with torch.no_grad():
    model.eval()
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    
# Save the trained model
torch.save(model.state_dict(), 'FNN_model.pt')

print(f"Accuracy: {accuracy:.2f}")


# Load the trained model
model = FeedforwardNN(input_dim, hidden_dim, output_dim)
model.load_state_dict(torch.load('FNN_model.pt'))
model.eval()

# Load the sentence transformer model
sentence_transformer_model = SentenceTransformer('bert-base-nli-mean-tokens')

# Load the embeddings and message encoding vocabulary
embeddings = torch.load('parsed_smell_identifier_embeddings.pt')
with open('message_encoding_vocabulary.pkl', 'rb') as f:
    message_encoding_vocabulary = pickle.load(f)

# Convert the embeddings to numpy array
existing_embeddings = np.array(embeddings.tolist())

# Load the input CSV file
input_data = pd.read_csv('input.csv', nrows=100)

# Extract the AST column
new_asts = input_data['AST'].tolist()

# Convert the new ASTs into embeddings
new_embeddings = sentence_transformer_model.encode(new_asts)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(new_embeddings).float()

# Perform prediction using the trained model
with torch.no_grad():
    model.eval()
    outputs = model(X_tensor)
    _, predicted = torch.max(outputs.data, 1)
    
# Calculate cosine similarity between new embeddings and existing embeddings
similarity_scores = cosine_similarity(new_embeddings, existing_embeddings)

# Perform prediction based on similarity threshold
for i in range(len(predicted)):
    predicted_index = predicted[i].item()
    predicted_message = message_encoding_vocabulary[y[predicted_index]]
    class_name = input_data.loc[i, 'class_name']
    line_number = input_data.loc[i, 'line_number']
    similarity = similarity_scores[i][predicted_index]

    print("Belonged Message:", predicted_message)
    print("Class Name:", class_name)
    print("Line Number:", line_number)
    print("Similarity Score:", similarity)
    print()



Accuracy: 0.76
Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 9
Similarity Score: 0.7796146783335485

Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 15
Similarity Score: 0.8998983777371008

Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 16
Similarity Score: 0.6767066002115315

Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 17
Similarity Score: 0.7695589910941011

Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 18
Similarity Score: 0.6767066002115315

Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 19
Similarity Score: 0.7507423865274395

Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 20
Similarity Score: 0.7438635941891709

Belonged Message: Adjustment
Class Name: 

# LSTM

In [87]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Load the embeddings from parsed_smell_identifier_embeddings.pt
embeddings = torch.load('parsed_smell_identifier_embeddings.pt')

# Load the encoded messages from messagelist.pkl
with open('messagelist.pkl', 'rb') as f:
    message_list = pickle.load(f)

# Convert the embeddings into a suitable format for comparison
existing_embeddings = np.array(embeddings.tolist())

# Create a DataFrame with the embeddings and encoded messages
data = pd.DataFrame({
    'parsed_smell_identifier_embeddings': existing_embeddings.tolist(),
    'message_encoded': message_list
})

# Prepare the data for the LSTM model
X = np.array(data['parsed_smell_identifier_embeddings'].tolist())
y = np.array(data['message_encoded'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model architecture
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])
        return output

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train).float()
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test).float()
y_test_tensor = torch.tensor(y_test)

# Reshape X_train_tensor and X_test_tensor to include the time steps dimension
X_train_tensor = X_train_tensor.unsqueeze(1)
X_test_tensor = X_test_tensor.unsqueeze(1)

# Create the LSTM model and move it to CPU
model = LSTMModel(X_train_tensor.shape[2], 128, len(np.unique(y_train)))
device = torch.device("cpu")
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader objects
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Training the LSTM model
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation on the test set
with torch.no_grad():
    model.eval()
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)

# Save the trained model
torch.save(model.state_dict(), 'LSTM_model.pt')
print(f"Accuracy: {accuracy:.2f}")


# Load the trained model
model = LSTMModel(input_dim, hidden_dim, output_dim)
model.load_state_dict(torch.load('LSTM_model.pt'))
model.eval()

# Load the sentence transformer model
sentence_transformer_model = SentenceTransformer('bert-base-nli-mean-tokens')

# Load the embeddings and message encoding vocabulary
embeddings = torch.load('parsed_smell_identifier_embeddings.pt')
with open('message_encoding_vocabulary.pkl', 'rb') as f:
    message_encoding_vocabulary = pickle.load(f)

# Convert the embeddings to numpy array
existing_embeddings = np.array(embeddings.tolist())

# Load the input CSV file
input_data = pd.read_csv('input.csv', nrows=100)

# Extract the AST column
new_asts = input_data['AST'].tolist()

# Convert the new ASTs into embeddings
new_embeddings = sentence_transformer_model.encode(new_asts)

# Calculate cosine similarity between new embeddings and existing embeddings
similarity_scores = cosine_similarity(new_embeddings, existing_embeddings)

# Perform prediction based on similarity threshold
for i in range(len(predicted)):
    predicted_index = predicted[i].item()
    predicted_message = message_encoding_vocabulary[y[predicted_index]]
    class_name = input_data.loc[i, 'class_name']
    line_number = input_data.loc[i, 'line_number']
    similarity = similarity_scores[i][predicted_index]

    print("Belonged Message:", predicted_message)
    print("Class Name:", class_name)
    print("Line Number:", line_number)
    print("Similarity Score:", similarity)
    print()



Accuracy: 0.77
Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 9
Similarity Score: 0.7796146783335485

Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 15
Similarity Score: 0.8998983777371008

Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 16
Similarity Score: 0.6767066002115315

Belonged Message: Rename
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 17
Similarity Score: 0.8262667464864443

Belonged Message: Remove
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 18
Similarity Score: 0.48644120674896285

Belonged Message: Adjustment
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 19
Similarity Score: 0.7507423865274395

Belonged Message: Rename
Class Name: ApplicationInsightsForK8sHostingStartup
Line Number: 20
Similarity Score: 0.7927418901252624

Belonged Message: Rename
Class Name: CGroupContainer

KeyError: 100